In [ ]:
# We use the following line to allow us to embed plots in Jupyter notebooks
%matplotlib notebook

# Load libraries. We need tensorflow and numpy for training the CNN,
# matplotlib for plotting, os.path and PIL for loading images,
# and random to partition the data into test and training sets
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import math
from os.path import dirname, abspath
from os.path import join
import os.path
import PIL
import random
import urllib.request
from PIL import Image, ImageEnhance, ImageStat

#seed random for putting images into training and test arrays
random.seed(13456)

In [ ]:
#location of the weights/biases and architecture for the CNN (saved above)
model_for_identification = "tmp/caffenet_pad_msh_tanzania.ckpt"
classes = 10
#keep_prob = tf.placeholder(tf.float32)


#called to identify subject in image
def identify(image):
    #reshape the image to an (1, 154587) vector
    image = np.mat(np.asarray(image).flatten())
    #create session
    with tf.Session() as sess:
        #load in the saved weights
        saver = tf.train.import_meta_graph(model_for_identification+'.meta')
        saver.restore(sess, model_for_identification)#tf.train.latest_checkpoint('tmp2/'))
        
        #get graph to extract tensors
        graph = tf.get_default_graph()
        pred = graph.get_tensor_by_name("pred:0")
        output = graph.get_tensor_by_name("output:0")
        X = graph.get_tensor_by_name("X:0")
        keep_prob = graph.get_tensor_by_name("keep_prob:0")

        #saver.restore(sess, model_for_identification)
        
        #find the prediction
        result = sess.run(pred, feed_dict={X: image, keep_prob: 1.})
        #we can look at the softmax output as well
        prob_array = sess.run(output, feed_dict={X: image, keep_prob: 1.})
        #print("op",prob_array)
        
        #return result (add 1 as indexed from 1 not 0)
        return result[0], prob_array

#pick an image, subject 1 to 39 and image 1 to 10, let's throw randoms to choose
test_drug = 8#random.randint(1,classes)
test_image = 74#random.randint(1,100)

#location of image.
image_name = 'msh_tanzania_data/test/4_31915.png'

#Load png file using the PIL library
im = PIL.Image.open(image_name)#join('images', str(test_drug) + '.' + str(test_image) + '.png'))

#catagorize
predicted_drug, predicted_prob = identify(im)

#load drug names
f = open('drug_names.txt')
firstline = f.readline()
drugs = firstline.split(',')
print(predicted_drug,drugs[predicted_drug], test_drug, test_image)

# #and print
# if predicted_drug == test_drug:
#     print("Drug",drugs[test_drug], "image", test_image, "\033[1mCORRECTLY\033[0m identified as drug", drugs[predicted_drug])
# else:
#     print("Drug",drugs[test_drug], "image", test_image, "\033[1mINCORRECTLY\033[0m identified as drug", drugs[predicted_drug])  

